# Dimensionality Reduction on MNIST: PCA & t-SNE

In [ ]:
%%capture
!sudo apt-get -y install libopenblas-dev liblapack-dev  # Ideally would put in a Sherlock environment
!sudo apt-get -y install libatlas-base-dev
!pip install git+https://github.com/alexisbcook/tsne.git
# !wget http://deeplearning.net/data/mnist/mnist.pkl.gz
!pip install hdbscan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import gzip, pickle

from sklearn.cluster import KMeans
import hdbscan
from sklearn.decomposition import PCA
from tsne import bh_sne

import seaborn as sns
palette = np.array(sns.color_palette("hls", 11))

%matplotlib inline  

The famous MNIST dataset is a collection of images of hand-written digits. A classic problem and now benchmmark for models in machine learning is whether it is possible to train a classifier to distinguish between these different digits. The MNIST dataset is actually labelled and so is appropriate for supervised learning but here we'll look at the extent to which purely unsupervised methods are able to segment it. In other words, we will only use the labels to check how well we've done at the end of the calculation. Each digit is represented as a (28x28) array of pixels and the dataset contains 70,000 images. 

In [ ]:
f = gzip.open("mnist.pkl.gz", "rb")
train, val, test = pickle.load(f, encoding='latin1')
f.close()

In [ ]:
X = np.asarray(np.vstack((train[0], val[0], test[0])), dtype=np.float64)
y = np.hstack((train[1], val[1], test[1]))

In [ ]:
feat_cols = ['pixel'+ str(i) for i in range(X.shape[1])]
df = pd.DataFrame(X,columns=feat_cols)
df['label'] = y.astype(int)

It is instructive to view the first few of these MNIST digits. The function below plots the first thirty:

In [ ]:
fig = plt.figure( figsize=(16,7) )
for i in range(0,30):
    ax = fig.add_subplot(3,10,i+1)
    ax.matshow(df.loc[i,feat_cols].values.reshape((28,28)).astype(float))
plt.show()

### 1. Principal Component Analysis (PCA)

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df.loc[:, feat_cols])

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.scatter(pca_result[:,0], pca_result[:,1])
ax2.scatter(pca_result[:,0], pca_result[:,1], c=palette[df['label'].astype(np.int)])
ax1.set_xlabel('PCA1')
ax1.set_ylabel('PCA2')
ax2.set_xlabel('PCA1')
ax2.set_ylabel('PCA2')
ax2.set_title('Coloured by the True Labels')

### 2. K-means clustering of PCA results

In [ ]:
clf = KMeans(n_clusters = 10, init='k-means++', random_state = 42)
clf.fit(pca_result)
indices= clf.labels_
df['pca label'] = indices

In [ ]:
f = plt.figure(figsize=(16, 16))
ax = plt.subplot(aspect='equal')
sc = ax.scatter( pca_result[:,0], pca_result[:,1], c=palette[indices.astype(np.int)] )
ax.set_xlabel('PCA1')
ax.set_ylabel('PCA2')
ax.set_title('Coloured by K-means labels')

We now group the dataframe by cluster label. We will attempt to eyeball features within each cluster in order to assign a true digit label to each cluster. We can do this by plotting a sample of images from each clusters and looking at what digits actually appear (remember the pixel values are the features and we are allowed to use these - just not the true labels). You can experiment with this, varying cluster_number from 0-9 and trying to assign a label to each of the 10 clusters.

In [ ]:
dict_of_clusters = {k: v for k, v in df.groupby('pca label')}

cluster_number = 6
fig = plt.figure( figsize=(16,7) )
for i in range(0,50):
    ax = fig.add_subplot(5,10,i+1)
    cluster_frames = dict_of_clusters[cluster_number].loc[:, feat_cols]
    ax.matshow(cluster_frames.iloc[i].values.reshape((28,28)).astype(float))
plt.show()

In [ ]:
# Eyeballed dictionary between cluster labels and the true labels  - have a go at this
true_labels_map = {0:3, 1:4, 2:6, 3:1, 4:2, 5:0, 6:5, 7:9, 8:8, 9:7} 

In [ ]:
df['pca predicted'] = df.loc[:, 'pca label'].apply(lambda x: true_labels_map[x])

In [ ]:
mask = (df['label']==df['pca predicted'])
accuracy = (df.loc[mask].shape[0])/(df.shape[0])
print("Percentage Accuracy of PCA + kmeans unsupervised learning = ~", 100*accuracy) # You'll get about 35% for PCA

### 3. t-Distributed Stochastic Neighbour Embedding (t-SNE)

Since t-SNE can be quite slow to run, we load the results from a .csv file. The code to run the algorithm is shown below (commented). The first implementation is the one found at https://github.com/alexisbcook/tsne, whilst the latter is the sklearn implementation. Note that the former seems to be faster and also more able to handle large datasets than the sklearn implementation, however it does not have a functional random_state parameter so save your output (it will not be exactly reproducible as t-SNE is not deterministic).

In [ ]:
tsne_result = np.genfromtxt ('MNIST_large_tsne.csv', delimiter=",") # Generated from the bh_sne() code

In [ ]:
# Implementation from https://github.com/alexisbcook/tsne - takes of the order 20 minutes to run
# time_start = time.time()
# tsne_result = bh_sne(df.loc[:, feat_cols])
# print(  't-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start)  )

In [ ]:
# Implementation from sklearn - this implementation may crash on the full MNIST data (may need a subset)
# time_start = time.time()
# tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=300, random_state = 42) 
# tsne_result = tsne.fit_transform(df_sample.loc[:, feat_cols]) 
# print(  't-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start)  )

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.scatter(tsne_result[:,0], tsne_result[:,1])
ax2.scatter(tsne_result[:,0], tsne_result[:,1], c=palette[df['label'].astype(np.int)])
ax1.set_xlabel('tSNE1')
ax1.set_ylabel('tSNE2')
ax2.set_xlabel('tSNE1')
ax2.set_ylabel('tSNE2')
ax2.set_title('Coloured by the True Labels')

In [ ]:
clf = KMeans(n_clusters = 10, init='k-means++', random_state = 42)
clf.fit(tsne_result)
indices= clf.labels_
df['tsne label'] = indices

In [ ]:
# Plot these results
f = plt.figure(figsize=(16, 16))
ax = plt.subplot(aspect='equal')
sc = ax.scatter( tsne_result[:,0], tsne_result[:,1], c=palette[indices.astype(np.int)] )
ax.set_xlabel('tSNE1')
ax.set_ylabel('tSNE2')
ax.set_title('Coloured by K-means labels')

In [ ]:
dict_of_clusters = {k: v for k, v in df.groupby('tsne label')}

cluster_number = 8
fig = plt.figure( figsize=(16,7) )
for i in range(0,50):
    ax = fig.add_subplot(5,10,i+1)
    cluster_frames = dict_of_clusters[cluster_number].loc[:, feat_cols]
    ax.matshow(cluster_frames.iloc[i].values.reshape((28,28)).astype(float))
plt.show()

In [ ]:
true_labels_map = {0:6, 1:3, 2:1, 3:0, 4:4, 5:8, 6:2, 7:7, 8:5, 9:9}  

In [ ]:
df['tsne predicted'] = df.loc[:, 'tsne label'].apply(lambda x: true_labels_map[x])

In [ ]:
mask = (df['label']==df['tsne predicted'])
accuracy = (df.loc[mask].shape[0])/(df.shape[0])
print("Percentage Accuracy of tSNE + kmeans unsupervised learning = ~", 100*accuracy)

### 4. Pushing the Accuracy Higher: Hierarchical DBSCAN (HDBSCAN)

In [ ]:
clf = hdbscan.HDBSCAN(min_cluster_size=56)
clf.fit(tsne_result)
indices_hdbcsan = clf.labels_
df['HDBSCAN label'] = indices_hdbcsan

In [ ]:
len(np.unique(indices_hdbcsan)) # DBSCAN + HDBSCAN includes 'not assigned to any cluster' as an option - hence the 11 clusters

In [ ]:
f = plt.figure(figsize=(16, 16))
ax = plt.subplot(aspect='equal')
sc = ax.scatter( tsne_result[:,0], tsne_result[:,1], c=palette[indices_hdbcsan.astype(np.int)] )
ax.set_xlabel('tSNE1')
ax.set_ylabel('tSNE2')
ax.set_title('Coloured by HDBSCAN labels')

In [ ]:
# Drop points that do not belong to any cluster
mask = (df['HDBSCAN label'] != -1)
df_signal = df[mask]

In [ ]:
dict_of_clusters = {k: v for k, v in df_signal.groupby('HDBSCAN label')}

cluster_number = 7
fig = plt.figure( figsize=(16,7) )
for i in range(0,50):
    ax = fig.add_subplot(5,10,i+1)
    cluster_frames = dict_of_clusters[cluster_number].loc[:, feat_cols]
    ax.matshow(cluster_frames.iloc[i].values.reshape((28,28)).astype(float))
plt.show()

In [ ]:
true_labels_map = {0:0, 1:6, 2:7, 3:9, 4:5, 5:3, 6:1, 7:8, 8:4, 9:2}  # Cluster 3 contains (9,4) - can separate them by 
                                                                      # further perplexity tuning in tSNE

In [ ]:
df_signal['tsne HDBSCAN predicted'] = df_signal.loc[:, 'HDBSCAN label'].apply(lambda x: true_labels_map[x])

In [ ]:
mask = (df_signal['label']==df_signal['tsne HDBSCAN predicted'])
accuracy = (df_signal.loc[mask].shape[0])/(df_signal.shape[0])
print("Percentage Accuracy of tSNE + HDBSCAN unsupervised learning = ~", 100*accuracy)